In [16]:
import numpy as np
import json, requests
import pandas as pd
import random
import cmlapi
import os

### Overview

the notebook will:
1. take input format for python endpoint
2. extract model url and model access key
3. reading input data, will take a single sample of link/location combination 
4. then take a random sample of 25 contiguous rows
5. Convert this into json format through 'dataframe_to_json' function
6. Put all pieces together for json payload required by endpoint
7. Send prediction request
8. 'Unpack' response

In [17]:
window = 24

In [18]:
def dataframe_to_json(df):
    """Create a dictionary that will later be converted to a JSON object
    ensure that 
    """
    data = {
        'rx_gbs': df['rx_gbs'].iloc[-(window+1):].tolist(),
        'tx_gbs': df['tx_gbs'].iloc[-(window+1):].tolist(),
        'time': [df['time'].iloc[-1]],  # Only the last time entry
        'link': [df['link'].iloc[-1]],  # Only the last link entry
        'location': [df['location'].iloc[-1]]  # Only the last location entry
    }
    
    # build embedded dictionary step 1
    request_dict = {"request":data}

    # access key will be end point specific
    BackDict = {"accessKey":model_key}
    BackDict.update(request_dict)
    request_dict=BackDict
    
    return request_dict

In [19]:
df = pd.read_csv('data/netstats_hourly_4_3.csv')
#df.head()

In [20]:
model_name = input("Enter the name of the model: ")

Enter the name of the model:  LSTM-no-metrics


In [21]:
## Need to get the below prgramatically in the future iterations
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
target_model = client.list_all_models(search_filter=json.dumps({"name": model_name}))
model_key = target_model.models[0].access_key
model_url = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model")

Pick a random link / location

In [22]:
sample_df = df[['link','location']].drop_duplicates().sample()
link_pick = sample_df['link'].values[0]
location_pick = sample_df['location'].values[0]
print(sample_df)

     link   location
4376  ER1  Frankfurt


In [23]:
# test_df has only one combintation  of link_pick and location_pick
test_df = df[(df.link == link_pick) & (df.location == location_pick)]

In [24]:
# pick a random segment of size window + 1
start_row = np.random.randint(0, len(test_df) - (window+1))
random_segment = df.iloc[start_row:start_row + (window+1)]

In [25]:
# confirm size
random_segment.shape

(25, 6)

In [26]:
random_segment.head()

,link,location,link_loc,time,rx_gbs,tx_gbs
491,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 00:00:00,1.216804,1.863456
492,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 01:00:00,1.279152,1.432579
493,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 02:00:00,1.264059,1.795645
494,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 03:00:00,0.983256,1.412110
495,CenturyLink,US-DA3,CenturyLink_US-DA3,2024-04-01 04:00:00,1.065229,1.612427


In [27]:
# convert input to json
request_dict = dataframe_to_json(random_segment)

In [15]:
#test_inputs

Step below puts it into required payload for endpoint

In [46]:
# # build embedded dictionary step 1
# request_dict = {"request":test_inputs}

# # access key will be end point specific
# BackDict = {"accessKey":model_key}
# BackDict.update(request_dict)
# request_dict=BackDict

In [28]:
r = requests.post(model_url, data=json.dumps(request_dict), headers={'Content-Type': 'application/json'})

In [29]:
r.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:af1ca1b6-2438-44e1-910f-e06f3320a279/0a76967a-59b2-4390-a481-f5998560b746',
  'prediction': {'rx_bytes': [[0.8272794485092163,
     0.8741030693054199,
     0.8947427272796631,
     0.9079001545906067,
     1.013359546661377,
     1.1580333709716797]],
   'time': ['2024-04-02T01:00:00',
    '2024-04-02T02:00:00',
    '2024-04-02T03:00:00',
    '2024-04-02T04:00:00',
    '2024-04-02T05:00:00',
    '2024-04-02T06:00:00'],
   'tx_bytes': [[0.07302311062812805,
     0.06987149268388748,
     0.061288222670555115,
     0.04081371799111366,
     0.07086709886789322,
     0.16421855986118317]]},
  'uuid': 'fa906280-0138-4182-97c3-ec5cbd4e5875'},
 'ReplicaID': 'lstm-no-metrics-10-21-546d45d488-rvsf9',
 'Size': 662,
 'StatusCode': 200}

Get the model response

In [49]:
r.json()['response']['prediction']

{'rx_bytes': [[9.300664901733398,
   8.624772071838379,
   7.315581321716309,
   6.047887325286865,
   5.334909439086914,
   4.521298885345459]],
 'time': ['2024-04-05T16:00:00',
  '2024-04-05T17:00:00',
  '2024-04-05T18:00:00',
  '2024-04-05T19:00:00',
  '2024-04-05T20:00:00',
  '2024-04-05T21:00:00'],
 'tx_bytes': [[0.4562716782093048,
   -0.4851105213165283,
   -1.0172359943389893,
   -0.9375834465026855,
   -0.73171466588974,
   -1.0454868078231812]]}